In [ ]:
from lego_factory import LegoFactory
import ocp_vscode


factory = LegoFactory()
result = factory.make_rectangle(2, 6, 1)
# exporters.export(result, "plate2x4.stl")
ocp_vscode.show_object(result, clear=True)

AttributeError: 'LegoFactory' object has no attribute 'make_rectangule'